In [1]:
import pandas as pd
import lightgbm as lgb
from sklearn.model_selection import train_test_split
import os


/opt/anaconda3/lib/python3.11/site-packages/dask/dataframe/_pyarrow_compat.py:23: UserWarning: You are using pyarrow version 11.0.0 which is known to be insecure. See https://www.cve.org/CVERecord?id=CVE-2023-47248 for further details. Please upgrade to pyarrow>=14.0.1 or install pyarrow-hotfix to patch your current version.
  warnings.warn(


In [2]:
# Cargar dataset
df = pd.read_csv('../data/ventas_featurev4_cluster.csv', sep=',')

In [3]:
# 🕓 Convertir periodo a formato entero YYYYMM si está como string con fecha completa
if df['periodo'].dtype == 'object' and '-' in df['periodo'].iloc[0]:
    df['periodo'] = pd.to_datetime(df['periodo']).dt.strftime('%Y%m%d').astype(int)

In [4]:
# 🧼 Filtrar a datos históricos hasta 201912
df_train = df[df['periodo'] <= 20191231].copy()

In [5]:
# 🧠 Definir variables
TARGET = 'tn'
EXCLUIR = ['tn', 'periodo', 'descripcion', 'prod_cust', 'cluster_predicho']  # No se usan como input
FEATURES_CATEGORICAS = df.select_dtypes(['category', 'object']).columns.tolist()
FEATURES_CATEGORICAS = [col for col in FEATURES_CATEGORICAS if col in df.columns and col not in EXCLUIR]

In [6]:
print(df_train.columns.tolist())


['periodo', 'customer_id', 'product_id', 'plan_precios_cuidados', 'tn', 'cust_request_qty', 'cust_request_tn', 'cat1', 'cat2', 'cat3', 'brand', 'sku_size', 'descripcion', 'productos_estrella', 'mes', 'catastrofe', 'cliente_estrella', 'prod_cust', 'tn_lag_1', 'tn_lag_2', 'tn_lag_3', 'tn_lag_4', 'tn_lag_5', 'tn_lag_6', 'tn_lag_7', 'tn_lag_8', 'tn_lag_9', 'tn_lag_10', 'tn_lag_11', 'tn_lag_12', 'tn_lag_13', 'tn_lag_14', 'tn_lag_15', 'tn_lag_16', 'tn_lag_17', 'tn_lag_18', 'tn_lag_19', 'tn_lag_20', 'tn_lag_21', 'tn_lag_22', 'tn_lag_23', 'tn_lag_24', 'tn_lag_25', 'tn_lag_26', 'tn_lag_27', 'tn_lag_28', 'tn_lag_29', 'tn_lag_30', 'tn_lag_31', 'tn_lag_32', 'tn_lag_33', 'tn_lag_34', 'tn_lag_35', 'tn_lag_36', 'tn_mas_2', 'delta_tn_lag_1', 'delta_tn_lag_2', 'delta_tn_lag_3', 'delta_tn_lag_4', 'delta_tn_lag_5', 'delta_tn_lag_6', 'delta_tn_lag_7', 'delta_tn_lag_8', 'delta_tn_lag_9', 'delta_tn_lag_10', 'delta_tn_lag_11', 'delta_tn_lag_12', 'delta_tn_lag_13', 'delta_tn_lag_14', 'delta_tn_lag_15', 'delta

In [7]:
print(df_train['cluster_predicho'].value_counts())


cluster_predicho
0    5057408
4    3554049
2    2961130
5    2917318
3    1934861
1     310914
Name: count, dtype: int64


In [8]:
import optuna
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

In [9]:

# Diccionario para guardar modelos entrenados
modelos = {}

# Entrenamiento por clúster
for cluster_id in sorted(df_train['cluster_predicho'].unique()):
    print(f"\n🔄 Entrenando modelo para cluster {cluster_id}...")

    # Subconjunto del cluster
    df_cluster = df_train[df_train['cluster_predicho'] == cluster_id].copy()

    # Asegurar que las categóricas estén como 'category'
    for col in FEATURES_CATEGORICAS:
        if col in df_cluster.columns:
            df_cluster[col] = df_cluster[col].astype('category')

    # Definir variables input y target
    columnas_input = [col for col in df_cluster.columns if col not in EXCLUIR]
    X = df_cluster[columnas_input]
    y = df_cluster[TARGET]

    # Separar en train y validación
    X_train, X_val, y_train, y_val = train_test_split(
        X, y, test_size=0.2, random_state=42
    )

    # Dataset LightGBM
    lgb_train = lgb.Dataset(X_train, label=y_train, categorical_feature=FEATURES_CATEGORICAS, free_raw_data=False)
    lgb_val = lgb.Dataset(X_val, label=y_val, reference=lgb_train, categorical_feature=FEATURES_CATEGORICAS, free_raw_data=False)

    # Función objetivo para Optuna
    def objective(trial):
        param_grid = {
            'objective': 'regression',
            'metric': 'rmse',
            'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3),
            'max_depth': trial.suggest_int('max_depth', 6, 16),
            'num_leaves': trial.suggest_int('num_leaves', 20, 255),
            'min_data_in_leaf': trial.suggest_int('min_data_in_leaf', 20, 200),
            'subsample': trial.suggest_float('subsample', 0.5, 1.0),
            'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0),
            'feature_fraction': trial.suggest_float('feature_fraction', 0.5, 1.0),
            'lambda_l1': trial.suggest_float('lambda_l1', 0.0, 5.0),
            'lambda_l2': trial.suggest_float('lambda_l2', 0.0, 5.0),
            'verbose': -1
        }

        gbm = lgb.train(
            param_grid,
            lgb_train,
            valid_sets=[lgb_val],
            valid_names=['valid'],
            num_boost_round=1000,
            callbacks=[
                lgb.early_stopping(stopping_rounds=30),
                lgb.log_evaluation(period=0)
            ]
        )

        preds = gbm.predict(X_val, num_iteration=gbm.best_iteration)
        rmse = mean_squared_error(y_val, preds, squared=False)
        return rmse

    # Correr Optuna
    study = optuna.create_study(direction="minimize")
    study.optimize(objective, n_trials=30)

    # Entrenar modelo final con mejores hiperparámetros
    best_params = study.best_params
    best_params.update({
        'objective': 'regression',
        'metric': 'rmse',
        'verbose': -1
    })

    print(f"📌 Mejores hiperparámetros para cluster {cluster_id}:")
    for k, v in best_params.items():
        print(f"   {k}: {v}")

    model = lgb.train(
        best_params,
        lgb_train,
        valid_sets=[lgb_val],
        valid_names=['valid'],
        num_boost_round=1000,
        callbacks=[
            lgb.early_stopping(stopping_rounds=50),
            lgb.log_evaluation(period=100)
        ]
    )

    # Guardar modelo
    modelo_path = f'lightgbm_cluster_{cluster_id}.txt'
    model.save_model(modelo_path)
    modelos[cluster_id] = model
    print(f"✅ Modelo guardado en {modelo_path}")



🔄 Entrenando modelo para cluster 0...


[I 2025-07-02 00:22:56,582] A new study created in memory with name: no-name-b1314418-aad7-4ae5-8b09-1fb8a36b5d69


Training until validation scores don't improve for 30 rounds
Did not meet early stopping. Best iteration is:
[1000]	valid's rmse: 0.0481972


/opt/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
[I 2025-07-02 00:27:14,491] Trial 0 finished with value: 0.048197153100087715 and parameters: {'learning_rate': 0.09095123023319422, 'max_depth': 16, 'num_leaves': 100, 'min_data_in_leaf': 108, 'subsample': 0.5335626045644526, 'colsample_bytree': 0.7930595913868994, 'feature_fraction': 0.7661995389917307, 'lambda_l1': 3.6082010142687926, 'lambda_l2': 3.9393525947442605}. Best is trial 0 with value: 0.048197153100087715.


Training until validation scores don't improve for 30 rounds
Did not meet early stopping. Best iteration is:
[1000]	valid's rmse: 0.046784


...

Training until validation scores don't improve for 30 rounds
Early stopping, best iteration is:
[708]	valid's rmse: 0.00109727


/opt/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
[I 2025-07-02 09:25:24,810] Trial 28 finished with value: 0.001097272289710619 and parameters: {'learning_rate': 0.1201818984870786, 'max_depth': 11, 'num_leaves': 147, 'min_data_in_leaf': 56, 'subsample': 0.8751147562428059, 'colsample_bytree': 0.5398498753179266, 'feature_fraction': 0.9123205915141174, 'lambda_l1': 0.3298546413664763, 'lambda_l2': 3.457499342641211}. Best is trial 24 with value: 0.0010892238036200102.


Training until validation scores don't improve for 30 rounds
Early stopping, best iteration is:
[801]	valid's rmse: 0.00124085


/opt/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
[I 2025-07-02 09:26:07,988] Trial 29 finished with value: 0.0012408542627998763 and parameters: {'learning_rate': 0.09770799212620346, 'max_depth': 11, 'num_leaves': 105, 'min_data_in_leaf': 55, 'subsample': 0.8943057569987869, 'colsample_bytree': 0.660820194721474, 'feature_fraction': 0.8682860216856194, 'lambda_l1': 0.9391729602624839, 'lambda_l2': 3.426739994115603}. Best is trial 24 with value: 0.0010892238036200102.


📌 Mejores hiperparámetros para cluster 5:
   learning_rate: 0.1698996346774407
   max_depth: 13
   num_leaves: 227
   min_data_in_leaf: 72
   subsample: 0.9725103330538538
   colsample_bytree: 0.6346905256515123
   feature_fraction: 0.8689741116377997
   lambda_l1: 0.24063992917357932
   lambda_l2: 3.372609181039029
   objective: regression
   metric: rmse
   verbose: -1
Training until validation scores don't improve for 50 rounds
[100]	valid's rmse: 0.00126175
[200]	valid's rmse: 0.00115031
[300]	valid's rmse: 0.00111488
[400]	valid's rmse: 0.00110108
[500]	valid's rmse: 0.00109567
[600]	valid's rmse: 0.00109203
[700]	valid's rmse: 0.00109026
[800]	valid's rmse: 0.0010894
[900]	valid's rmse: 0.00108841
[1000]	valid's rmse: 0.00108765
Did not meet early stopping. Best iteration is:
[1000]	valid's rmse: 0.00108765
✅ Modelo guardado en lightgbm_cluster_5.txt


In [10]:
# 🧱 Paso 1: Crear base para 202002 copiando la de 201912
df_pred = df[df['periodo'] == 20191201].copy()
df_pred['periodo'] = 20200201  # Modificar período


In [11]:
# 🧠 Paso 2: Cargar modelos por cluster
modelos = {}
for cluster_id in df_train['cluster_predicho'].unique():
    path = f'lightgbm_cluster_{cluster_id}.txt'
    modelos[cluster_id] = lgb.Booster(model_file=path)


In [12]:
print(len(df_pred))

465660


In [13]:
# 🎯 Paso 3: Predecir por cluster
predicciones = []
for cluster_id in sorted(df_pred['cluster_predicho'].unique()):
    print(f"🔮 Prediciendo cluster {cluster_id}...")
    df_cluster = df_pred[df_pred['cluster_predicho'] == cluster_id].copy()

    # Asegurar categorías
    for col in FEATURES_CATEGORICAS:
        if col in df_cluster.columns:
            df_cluster[col] = df_cluster[col].astype('category')

    columnas_input = [col for col in df_cluster.columns if col not in EXCLUIR]
    X_pred = df_cluster[columnas_input]
    modelo = modelos[cluster_id]
    df_cluster['pred_tn'] = modelo.predict(X_pred)
    predicciones.append(df_cluster)




🔮 Prediciendo cluster 0...
🔮 Prediciendo cluster 1...
🔮 Prediciendo cluster 2...
🔮 Prediciendo cluster 3...
🔮 Prediciendo cluster 4...
🔮 Prediciendo cluster 5...


In [14]:
# ✅ Paso final: unir predicciones, agrupar por product_id y exportar
if predicciones:
    df_resultado = pd.concat(predicciones)

    # Agrupar por product_id y sumar las toneladas predichas
    df_export = (
        df_resultado.groupby('product_id', as_index=False)['tn']
        .sum()
        .sort_values('product_id')  # opcional, para orden prolijo
    )

    # Exportar archivo final
    df_export.to_csv('../data/predicciones_202002_clusterv2.csv', index=False)
    print("✅ Archivo guardado: predicciones_202002_clusterv2.csv")
else:
    print("⚠️ No se generó ninguna predicción.")


✅ Archivo guardado: predicciones_202002_clusterv2.csv
